#FactorCL on Synthetic Dataset

In [1]:
!git clone https://github.com/pliang279/FactorCL

Cloning into 'FactorCL'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 150 (delta 13), reused 2 (delta 0), pack-reused 116
Receiving objects: 100% (150/150), 321.78 KiB | 3.04 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [2]:
%cd FactorCL

/home/jovyan/work/FactorCL_CBM/Synthetic/FactorCL


In [3]:
from Synthetic.dataset import*
from synthetic_model import*
from torch.utils.data import DataLoader
from sklearn.linear_model import LogisticRegression

In [4]:
hidden_dim=512
embed_dim=128

In [5]:

# Define custom dimensions of features and labels
feature_dim_info = dict()
label_dim_info = dict()

intersections = get_intersections(num_modalities=2)

feature_dim_info['12'] = 10
feature_dim_info['1'] = 6
feature_dim_info['2'] = 6

label_dim_info['12'] = 10
label_dim_info['1'] = 6
label_dim_info['2'] = 6

print(intersections)
print(feature_dim_info)
print(label_dim_info)

# Get datasets
total_data, total_labels, total_raw_features = generate_data(30000, 2, feature_dim_info, label_dim_info)
total_labels = get_labels(label_dim_info, total_raw_features)

dataset = MultimodalDataset(total_data, total_labels)

# Dataloader
batch_size = 256
num_data = total_labels.shape[0]
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.8*num_data), num_data-int(0.8*num_data)])

train_loader = DataLoader(train_dataset, shuffle=True, drop_last=True,
                            batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=False, drop_last=False,
                            batch_size=batch_size)
data_loader = DataLoader(dataset, shuffle=False, drop_last=False,
                            batch_size=batch_size)


['1', '2', '12']
{'12': 10, '1': 6, '2': 6}
{'12': 10, '1': 6, '2': 6}


##Linear Probing

###FactorCL-SUP

In [6]:
factorcl_sup = FactorCLSUP(A_dim, B_dim, 20, hidden_dim, embed_dim).cuda()
train_sup_model(factorcl_sup, train_loader, dataset, num_epoch=10, num_club_iter=1)
factorcl_sup.eval()

NameError: name 'A_dim' is not defined

In [ ]:
# Simple evaluation using linear logistic regression model

# Embeddings
train_embeds = factorcl_sup.get_embedding(torch.stack(train_dataset[:][:-1]).cuda()).detach().cpu().numpy()
train_labels = train_dataset[:][-1].detach().cpu().numpy()

test_embeds = factorcl_sup.get_embedding(torch.stack(test_dataset[:][:-1]).cuda()).detach().cpu().numpy()
test_labels = test_dataset[:][-1].detach().cpu().numpy()

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

In [ ]:
score

###FactorCL-SSL

In [ ]:
factorcl_ssl = FactorCLSSL(A_dim, B_dim, hidden_dim, embed_dim).cuda()
train_sup_model(factorcl_ssl, train_loader, dataset, num_epoch=10, num_club_iter=1)
factorcl_ssl.eval()

In [ ]:
# Simple evaluation using linear logistic regression model

# Embeddings
train_embeds = factorcl_ssl.get_embedding(torch.stack(train_dataset[:][:-1]).cuda()).detach().cpu().numpy()
train_labels = train_dataset[:][-1].detach().cpu().numpy()

test_embeds = factorcl_ssl.get_embedding(torch.stack(test_dataset[:][:-1]).cuda()).detach().cpu().numpy()
test_labels = test_dataset[:][-1].detach().cpu().numpy()

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

In [ ]:
score

###SimCLR

In [ ]:
# set use_label=True for SupCon model
simclr_model = SupConModel(A_dim, B_dim, hidden_dim, embed_dim, use_label=False).cuda()
simclr_optim = optim.Adam(simclr_model.parameters(), lr=lr)
train_supcon(simclr_model, train_loader, simclr_optim, num_epoch=20)
simclr_model.eval()

In [ ]:
train_embeds = simclr_model.get_embedding(torch.stack(train_dataset[:][:-1]).cuda()).detach().cpu().numpy()
train_labels = train_dataset[:][-1].detach().cpu().numpy()

# Embeddings
test_embeds = simclr_model.get_embedding(torch.stack(test_dataset[:][:-1]).cuda()).detach().cpu().numpy()
test_labels = test_dataset[:][-1].detach().cpu().numpy()

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

In [ ]:
score

#MI Estimation

We can estimate the MI between any two sets of vectors using the Probablistic Classifier approach proposed in [Neural Methods for Point-wise Dependency Estimation](https://arxiv.org/abs/2006.05553)

In [ ]:
# Simple dataset for constructing paired data

class PairedDataset(Dataset):
  def __init__(self, data_A, data_B):
    self.data_A = data_A.float()
    self.data_B = data_B.float()

  def __len__(self):
    return self.data_A.shape[0]

  def __getitem__(self, idx):
    return self.data_A[idx], self.data_B[idx]

  def get_dims(self):
    return self.data_A.shape[1], self.data_B.shape[1]


def train_critic_paired(critic_model, train_loader, opt_crit, num_epoch=80):
    MIs = []
    for _iter in range(num_epoch):
        for i_batch, (A_batch, B_batch) in enumerate(train_loader):
            opt_crit.zero_grad()

            x_batch = A_batch.cuda()
            y_batch = B_batch.cuda()

            scores = critic_model(x_batch, y_batch)
            MIs.append(probabilistic_classifier_eval(scores))

            negative_loss = probabilistic_classifier_obj(scores)
            loss = -negative_loss

            loss.backward()
            opt_crit.step()

            if i_batch%100 == 0:
                print('iter: ', _iter, ' i_batch: ', i_batch, ' negative_loss: ', negative_loss.item())

    return MIs


def eval_MI_paired(critic_model, test_loader):
    MIs = []

    for i_batch, (A_batch, B_batch) in enumerate(test_loader):

        x_batch = A_batch.cuda()
        y_batch = B_batch.cuda()

        scores = critic_model(x_batch, y_batch)
        MIs.append(probabilistic_classifier_eval(scores))

    MI = torch.stack(MIs).mean()
    print(MI.item())
    return MI.item()


In [ ]:
layers = 1
activation = 'relu'
lr = 1e-4

In [ ]:
data_A = train_dataset[:][0]
data_B = train_dataset[:][1]

mi_dataset = PairedDataset(data_A, data_B)
mi_loader = DataLoader(mi_dataset,
                                 shuffle=True,
                                 batch_size=batch_size)

dim1, dim2 = mi_dataset.get_dims()
critic_model = SeparableCritic(x1_dim=dim1, x2_dim=dim2, hidden_dim=hidden_dim,
                              embed_dim=embed_dim, layers=layers, activation=activation).cuda()

opt_crit = optim.Adam(critic_model.parameters(), lr=lr)

train_critic_paired(critic_model, mi_loader, opt_crit, num_epoch=30)
MI = eval_MI_paired(critic_model, mi_loader)


